In [1]:
# -*- coding: utf-8 -*-
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

EPISODES = 1000


class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

/home/peronio/Dev/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
# agent.load("./save/cartpole-dqn.h5")
done = False
batch_size = 32

for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    print(state)
    for time in range(500):
        # env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        print(state)
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, EPISODES, time, agent.epsilon))
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
    # if e % 10 == 0:
#     agent.save("./save/cartpole-dqn.h5")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[[ 0.02074569  0.00744909 -0.02348436 -0.0493002 ]]
[[ 0.02089468  0.20289978 -0.02447036 -0.34929915]]
[[ 0.02495267  0.00813424 -0.03145634 -0.06443189]]
[[ 0.02511536  0.20369275 -0.03274498 -0.36687116]]
[[ 0.02918921  0.39926434 -0.04008241 -0.66969659]]
[[ 0.0371745   0.59491999 -0.05347634 -0.97472542]]
[[ 0.0490729   0.40055459 -0.07297085 -0.69930865]]
[[ 0.05708399  0.59660831 -0.08695702 -1.01404101]]
[[ 0.06901616  0.79277569 -0.10723784 -1.33271344]]
[[ 0.08487167  0.98907381 -0.13389211 -1.65693672]]
[[ 0.10465315  1.18547945 -0.16703084 -1.98815515]]
[[ 0.12836274  0.99245791 -0.20679395 -1.75153137]]
[[ 0.14821189  1.1892402  -0.24182457 -2.10078094]]
episode: 0/1000, score: 11, e: 1.0
[[ 0.03849889 -0.03459015 -0.03486893  0.02187725]]
[[ 0.03780709 -0.22919514 -0.03443138  0.30335792]]
[[ 0.03322319 -0.4238099  -0.02836422  0.58498621]]
[[ 0.02474699 -0.22830236 -0.01666

KeyboardInterrupt: 